In [11]:
import pandas as pd

# f = pd.read_json(file_path, lines=True)
df_image_train = pd.read_json("data/train.jsonl", lines=True)
df_image_val = pd.read_json("data/dev.jsonl", lines=True)
df_image_test = pd.read_json("data/test.jsonl", lines=True)
# df_image_val.columns

Index(['id', 'img', 'text'], dtype='object')

In [4]:
!pip install torch
!pip install transformers
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid 

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.37173524e-02 -4.28515933e-02 -1.56286098e-02  1.40537601e-02
  3.95537615e-02  1.21796273e-01  2.94333510e-02 -3.17523740e-02
  3.54959592e-02 -7.93140307e-02  1.75878126e-02 -4.04369533e-02
  4.97259796e-02  2.54913010e-02 -7.18699992e-02  8.14968497e-02
  1.47073215e-03  4.79627401e-02 -4.50335406e-02 -9.92174894e-02
 -2.81769633e-02  6.45045787e-02  4.44670655e-02 -4.76217344e-02
 -3.52952220e-02  4.38671820e-02 -5.28566204e-02  4.33004228e-04
  1.01921462e-01  1.64072476e-02  3.26996781e-02 -3.45987007e-02
  1.21339587e-02  7.94871375e-02  4.58342675e-03  1.57778431e-02
 -9.68208257e-03  2.87626348e-02 -5.05806394e-02 -1.55793848e-02
 -2.87907179e-02 -9.62278340e-03  3.15556489e-02  2.27349754e-02
  8.71449485e-02 -3.85027528e-02 -8.84718522e-02 -8.75496119e-03
 -2.12343484e-02  2.08924413e-02 -9.02078077e-02 -5.25732227e-02
 -1.05638560e-02  2.88311280e-02 -1.61455050e-02  6.17836602e-03
 -1.23234

In [20]:
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
from transformers import AdamW
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

# Load the sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Assume we have some data
# texts = ['This is a good day!', 'I feel very sad', 'I am not happy about this', 'I am thrilled']
# labels = [1, 0, 0, 1]  # 1: Positive sentiment, 0: Negative sentiment
X_train = df_image_train['text']
y_train = df_image_train['label']

X_val = df_image_val['text']
y_val = df_image_val['label']

# X_test = df_image_test['text']
# y_test = df_image_test['label']

# # Split into training and testing sets
# train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

# Convert the texts to embeddings
train_embeddings = model.encode(X_train)
test_embeddings = model.encode(X_val)

# Convert everything into torch tensors
train_embeddings = torch.tensor(train_embeddings)
train_labels = torch.tensor(y_train)

test_embeddings = torch.tensor(test_embeddings)
test_labels = torch.tensor(y_val)

# A simple linear classifier on top of SBERT's embeddings
classifier = nn.Linear(train_embeddings.size(1), 2)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = AdamW(classifier.parameters(), lr=1e-2)

# Training loop
for epoch in range(300):  # for simplicity, we train for 10 epochs
    classifier.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = classifier(train_embeddings)
    loss = criterion(outputs, train_labels)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Test the classifier
classifier.eval()

with torch.no_grad():
    outputs = classifier(test_embeddings)
    _, predicted = torch.max(outputs, 1)

print('Accuracy: ', (predicted == test_labels).sum().item() / test_labels.size(0))

Epoch 1, Loss: 0.6876366138458252
Epoch 2, Loss: 1.3108526468276978
Epoch 3, Loss: 0.7335209250450134
Epoch 4, Loss: 0.8349707722663879
Epoch 5, Loss: 0.9584453701972961
Epoch 6, Loss: 0.7120893001556396
Epoch 7, Loss: 0.611566424369812
Epoch 8, Loss: 0.7412387728691101
Epoch 9, Loss: 0.7872076630592346
Epoch 10, Loss: 0.6958101391792297
Epoch 11, Loss: 0.6032505035400391
Epoch 12, Loss: 0.6378868222236633
Epoch 13, Loss: 0.6974227428436279
Epoch 14, Loss: 0.6537891626358032
Epoch 15, Loss: 0.5836477279663086
Epoch 16, Loss: 0.5888790488243103
Epoch 17, Loss: 0.6355252861976624
Epoch 18, Loss: 0.6444496512413025
Epoch 19, Loss: 0.6063341498374939
Epoch 20, Loss: 0.5740800499916077
Epoch 21, Loss: 0.587197482585907
Epoch 22, Loss: 0.6090907454490662
Epoch 23, Loss: 0.5930168032646179
Epoch 24, Loss: 0.5661014318466187
Epoch 25, Loss: 0.5713928937911987
Epoch 26, Loss: 0.5897137522697449
Epoch 27, Loss: 0.5849149227142334
Epoch 28, Loss: 0.5637670755386353
Epoch 29, Loss: 0.5585206747055

In [23]:
# with torch.no_grad():
#     outputs = classifier(train_embeddings)
#     _, predicted = torch.max(outputs, 1)

# print('Accuracy: ', (predicted == train_labels).sum().item() / test_labels.size(0))